In [1]:
test_dict = {"항공권":[{"subMenuKey":"sub-menu 항공권서브 메뉴1","subMenuValue":"항공권서브 메뉴1옵션1"},{"subMenuKey":"sub-menu 항공권서브 메뉴2","subMenuValue":"항공권서브 메뉴2옵션2"}],
"렌트카":[{"subMenuKey":"sub-menu 렌트카서브 메뉴1","subMenuValue":"렌트카서브 메뉴1옵션2"}],
"호텔":[{"subMenuKey":"sub-menu 호텔서브 메뉴2","subMenuValue":"호텔서브 메뉴2옵션3"}],
"dateRange":"07/25/2023 - 07/25/2023"}

In [12]:
from datetime import datetime

def unfoil_inner_json(json_data:list):
    'json 안에 호텔,렌트카,항공권 개별 데이터 처리'
    # 솔직히 리스트였으면 아래 스텝 하나 필요 없음
    list_ = [list(item.values()) for item in json_data] # [[칼럼명,값],...]
    list_ = [' = :'.join(item) for item in list_] # ['칼럼명 = :값',...]
    return ' AND '.join(list_) # '칼럼명1 = :값1 AND 칼럼명2 =: 값2,...'

def unfoil_outer_json(json_data:dict):
    '전체 json 데이터 정리'
    dict_ = {
        '항공권': None,
        '렌트카': None,
        '호텔': None
    }
    for idx, (k, v) in enumerate(json_data.items()):
        # 항공권, 렌트카, 호텔이라면
        if idx != 3:
            # str 저장
            dict_[k] = unfoil_inner_json(v)
        # 날짜라면
        else:
            # 날짜 추가해야함
            start_date, end_date = v.split(' - ')
            start_date = datetime.strptime(start_date, r'%m/%d/%Y').strftime(r'%Y-%m-%d %H:%M:%S')
            end_date = datetime.strptime(end_date, r'%m/%d/%Y').strftime(r'%Y-%m-%d %H:%M:%S')
            where_date_part = f' AND start_date = :{start_date} AND end_date = :{end_date}'
            for k_ in dict_:
                # 날짜 추가
                dict_[k_] += where_date_part
    return dict_
        
dict_ = unfoil_outer_json(test_dict)

# 1. 렌트카
sql_select_cars = f"""
SELECT *
FROM cars
WHERE
  {dict_['렌트카']}
ORDER BY ratings DESC
LIMIT 3
"""

# 2. 호텔
sql_select_hotels = f"""
SELECT *
FROM hotels
WHERE
  {dict_['호텔']}
ORDER BY ratings DESC
LIMIT 3
"""

# sql 내에서 계산할 것이 아니라 python에서 계산한다면
# 가는 표 오는 표 각각 필요할 듯
# 3. 항공권
flight_where_to_jeju = dict_['항공권'] + ' AND arrival = :CJU'
flight_where_from_jeju = dict_['항공권'] + ' AND departure = :CJU'
sql_select_flights_to_jeju = f"""
SELECT *
FROM hotels
WHERE
  {flight_where_to_jeju}
ORDER BY ratings DESC
LIMIT 3
"""
sql_select_flights_from_jeju = f"""
SELECT *
FROM hotels
WHERE
  {flight_where_from_jeju}
ORDER BY ratings DESC
LIMIT 3
"""

print(sql_select_cars)
print(sql_select_hotels)
print(sql_select_flights_to_jeju)
print(sql_select_flights_from_jeju)

# sql 내에서 계산까지 한다면 아래 참고
# # 3. 항공권 
# sql_query3 = f"""
# SELECT 
#     sub.name, 
#     sub.seat, 
#     sub.adult_charge, 
#     sub.leavehour, 
#     sub.departure, 
#     sub.arrival, 
#     sub.departure_datetime, 
#     sub.arrival_datetime, 
#     sub.check, 
#     sub.child_charge, 
#     sub.departure_kor, 
#     sub.arrival_kor,
#     sub.adult_charge * :num_adults + sub.child_charge * :num_children AS total_price
# FROM 
#     (SELECT 
#         flights.name, 
#         flights.seat, 
#         flights.adult_charge, 
#         flights.leavehour, 
#         flights.departure, 
#         flights.arrival, 
#         flights.departure_datetime, 
#         flights.arrival_datetime, 
#         flights.check, 
#         flights.child_charge, 
#         flights.departure_kor, 
#         flights.arrival_kor
#     FROM 
#         flights 
#     WHERE 
#         flights.departure_kor = :departure_kor AND
#         flights.arrival_kor = :arrival_kor AND
#         flights.금전상황 = :금전상황 AND
#         {date_keys3[0]} = :departure_datetime AND
#         {date_keys3[1]} = :arrival_datetime
#     ORDER BY 
#         flights.adult_charge 
#     LIMIT 3) AS sub
# """



SELECT *
FROM cars
WHERE
  sub-menu 렌트카서브 메뉴1 = :렌트카서브 메뉴1옵션2 AND start_date = :2023-07-25 00:00:00 AND end_date = :2023-07-25 00:00:00
ORDER BY ratings DESC
LIMIT 3


SELECT *
FROM hotels
WHERE
  sub-menu 호텔서브 메뉴2 = :호텔서브 메뉴2옵션3 AND start_date = :2023-07-25 00:00:00 AND end_date = :2023-07-25 00:00:00
ORDER BY ratings DESC
LIMIT 3


SELECT *
FROM hotels
WHERE
  sub-menu 항공권서브 메뉴1 = :항공권서브 메뉴1옵션1 AND sub-menu 항공권서브 메뉴2 = :항공권서브 메뉴2옵션2 AND start_date = :2023-07-25 00:00:00 AND end_date = :2023-07-25 00:00:00 AND arrival = :CJU
ORDER BY ratings DESC
LIMIT 3


SELECT *
FROM hotels
WHERE
  sub-menu 항공권서브 메뉴1 = :항공권서브 메뉴1옵션1 AND sub-menu 항공권서브 메뉴2 = :항공권서브 메뉴2옵션2 AND start_date = :2023-07-25 00:00:00 AND end_date = :2023-07-25 00:00:00 AND departure = :CJU
ORDER BY ratings DESC
LIMIT 3



In [ ]:
def get_connection():
    'connection obj return하는 함수'
    return

def fetch_data(sql_select)->list:
    'DB에서 데이터 fetch하기'
    conn = get_connection()
    cur = conn.cursor()

    cur.execute(sql_select)

    rows = cur.fetchall()

    cur.close()
    conn.close()

    return rows

cars_top3 = fetch_data(sql_select_cars)
hotels_top3 = fetch_data(sql_select_hotels)
flights_to_jeju_top3 = fetch_data(sql_select_flights_to_jeju)
flights_from_jeju_top3 = fetch_data(sql_select_flights_from_jeju)

In [5]:
def update_json_dates(json_data, selected_dates):
    for category in json_data:
        for i, item in enumerate(json_data[category]):
            for key in item:
                # If the key is a date key, update the value with the selected date
                if 'date' in key or 'datetime' in key:
                    json_data[category][i][key] = selected_dates[key]

selected_dates = {
    "start_date": "2023-01-05",
    "end_date": "2023-01-15",
    "departure_datetime": "2023-01-05",
    "arrival_datetime": "2023-01-15"
}
update_json_dates(json_data, selected_dates)

print(json_data)


{'렌트카': [{'start_date': '2023-01-05'}, {'end_date': '2023-01-15'}], '항공권': [{'departure_datetime': '2023-01-05'}, {'arrival_datetime': '2023-01-15'}], '호텔': [{'start_date': '2023-01-05'}, {'end_date': '2023-01-15'}]}


In [ ]:
'''
< 렌트카 SQL >
- cars 라는 테이블 가정
- choice 라는 금전 상황 컬럼 가정
- :selected_seats, :selected_size, :selected_transmission_type, :selected_age_req, :selected_driving_experience, :selected_choice, :selected_start_date, :selected_end_date 은 사용자가 선택해야할 값

SELECT *
FROM cars
WHERE
  seats = :selected_seats AND
  size = :selected_size AND
  transmission_type = :selected_transmission_type AND
  age_req = :selected_age_req AND
  driving_experice = :selected_driving_experience AND
  choice = :selected_choice AND
  start_date = :selected_start_date AND
  end_date = :selected_end_date
ORDER BY ratings DESC
LIMIT 3
'''

In [ ]:
''' 
< 호텔 SQL >
- hotels 라는 테이블 가정
- choice 라는 금전 상황 컬럼 가정
- :selected_region, :selected_capacity, :selected, choice 은 사용자가 선택해야할 값

SELECT *
FROM 
    hotels 
INNER JOIN 
    cars ON DATE(hotels.start_date) = DATE(cars.start_date) AND DATE(hotels.end_date) = DATE(cars.end_date)
WHERE 
    hotels.region = :selected_region AND
    hotels.capacity = :selected_capacity AND
    hotels.choice = :selected choice 
ORDER BY 
    hotels.ratings DESC 
LIMIT 3
'''

In [ ]:
'''
< 항공권 SQL >
- flights 라는 테이블 가정
- choice 라는 금전 상황 컬럼 가정
- :num_adults, :num_children, :departure_kor, :arrival_kor, choice 은 사용자가 선택해야할 값


SELECT 
    sub.name, 
    sub.seat, 
    sub.adult_charge, 
    sub.leavehour, 
    sub.departure, 
    sub.arrival, 
    sub.departure_datetime, 
    sub.arrival_datetime, 
    sub.check, 
    sub.child_charge, 
    sub.departure_kor, 
    sub.arrival_kor,
    sub.adult_charge * :num_adults + sub.child_charge * :num_children AS total_price
FROM 
    (SELECT 
        flights.name, 
        flights.seat, 
        flights.adult_charge, 
        flights.leavehour, 
        flights.departure, 
        flights.arrival, 
        flights.departure_datetime, 
        flights.arrival_datetime, 
        flights.check, 
        flights.child_charge, 
        flights.departure_kor, 
        flights.arrival_kor
    FROM 
        flights 
    INNER JOIN 
        cars ON DATE(flights.departure_datetime) = DATE(cars.start_date) AND DATE(flights.arrival_datetime) = DATE(cars.end_date)
    WHERE 
        flights.departure_kor = :departure_kor AND
        flights.arrival_kor = :arrival_kor AND
        flights.choice = :choice 
    ORDER BY 
        flights.adult_charge 
    LIMIT 3) AS sub
'''